In [113]:
import pandas as pd

In [114]:
NOME_TABELA_UNICA = '1.4-Filmes'

In [115]:
#Lendo o arquivo de dados
df = pd.read_csv('../dados.csv',dtype='object')
df.head()

,idFilme,Nome,Gênero,idMidia,Idioma,Tipo,Seção,Preco
0,656565,Uma janela suspeita,Drama,1001,dub,DVD,25,"3,5"
1,656565,Uma janela suspeita,Drama,1002,dub,VHS,25,"3,5"
2,656565,Uma janela suspeita,Drama,1003,leg,DVD,25,"3,5"
3,656565,Uma janela suspeita,Drama,1004,leg,VHS,25,"3,5"
4,323232,Minority Report,Ficção,2550,dub,DVD,32,"4,2"


In [116]:
serie_filmes = df.apply(lambda x: x.to_dict(),axis=1)

In [117]:
serie_filmes.get(0)

{'idFilme': '656565',
 'Nome': 'Uma janela suspeita',
 'Gênero': 'Drama',
 'idMidia': '1001',
 'Idioma': 'dub',
 'Tipo': 'DVD',
 'Seção': '25',
 'Preco': '3,5'}

In [118]:
lista_familia_colunas = ['Gênero','Idioma','Tipo']

In [119]:
def gerar_lista_e_colunas(nome_coluna):
    lista_unicos = df[nome_coluna].unique().tolist()
    lista_colunas = df.columns.tolist()
    lista_colunas.remove(nome_coluna)
    lista_colunas.remove('idMidia')
    return lista_unicos,lista_colunas

In [120]:
dict_familias = {}
for familia_coluna in lista_familia_colunas:
    familia, lista_colunas = gerar_lista_e_colunas(familia_coluna)
    dict_familias[familia_coluna]= [familia,lista_colunas]
print(dict_familias)

{'Gênero': [['Drama', 'Ficção'], ['idFilme', 'Nome', 'Idioma', 'Tipo', 'Seção', 'Preco']], 'Idioma': [['dub', 'leg'], ['idFilme', 'Nome', 'Gênero', 'Tipo', 'Seção', 'Preco']], 'Tipo': [['DVD', 'VHS'], ['idFilme', 'Nome', 'Gênero', 'Idioma', 'Seção', 'Preco']]}


In [121]:
def criar_put(tabela,rowkey,familia,coluna,valor):
    return "put '"+tabela+"','"+rowkey+"','"+familia+":"+coluna+"','"+valor+"'"

# Script criação

In [122]:
lista_familias = []
for item in dict_familias.values():
    lista_familias += item[0]

In [123]:
script_create  = "create '"+NOME_TABELA_UNICA+"','"+"','".join(lista_familias)+"'"

In [124]:
lista_put = []
for filme in serie_filmes:
    row_key = filme['idMidia']
    for nome_coluna,value in dict_familias.items():
        fc = value[0]
        fqs = value[1]
        familia = filme[nome_coluna]
        for fq in fqs:
            lista_put.append("put '"+NOME_TABELA_UNICA+"','"+row_key+"','"+familia+":"+fq+"','"+filme[fq]+"'")

In [125]:
lista_scripts = [script_create] +lista_put

In [126]:
with open('popular_modelo.txt', 'w') as f:
    for item in lista_scripts:
        f.write("%s\n" % item)